In [26]:
import pandas as pd
import requests

In [3]:
df = pd.read_csv('data/final_data/combined_data.csv')

print('Dataframe Print')
display(df)

Dataframe Print


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,3510255EC224A971,electric_bike,2023-08-01 00:37:40,2023-08-01 00:53:19,Broadway & W 58 St,6948.1,Morningside Dr & Amsterdam Ave,7741.04,40.766953,-73.981693,40.810285,-73.957365,casual
1,4F68D8A99EDAC92B,classic_bike,2023-08-01 01:15:28,2023-08-01 01:30:44,Allen St & Hester St,5342.1,John St & William St,5065.04,40.716009,-73.991952,40.708621,-74.007222,member
2,E5005E70D0099A96,classic_bike,2023-08-01 02:57:52,2023-08-01 03:19:44,W 106 St & Amsterdam Ave,7634.01,Madison Ave & E 120 St,7667.04,40.800836,-73.966449,40.802950,-73.942954,member
3,9010CDC8C004135E,classic_bike,2023-08-01 03:14:09,2023-08-01 03:19:27,W 27 St & 7 Ave,6247.06,Lexington Ave & E 26 St,6089.08,40.746647,-73.993915,40.741459,-73.983293,member
4,E526EF90BBD8CB0F,classic_bike,2023-08-01 04:41:52,2023-08-01 04:48:44,44 Dr & Jackson Ave,6352.01,36 Ave & 31 St,6572.08,40.747067,-73.943227,40.756513,-73.929575,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,33822DF82701AEBC,classic_bike,2023-07-31 19:24:53,2023-07-31 19:31:26,W 76 St & Columbus Ave,7281.09,2 Ave & E 72 St,6925.09,40.780304,-73.977716,40.768762,-73.958408,member
16770,FDB8A246C88C25A5,classic_bike,2023-07-31 20:56:19,2023-07-31 21:05:05,2 Ave & E 122 St,7622.12,2 Ave & E 122 St,7622.12,40.800436,-73.934936,40.800672,-73.934900,member
16771,18973BCA05D7B153,classic_bike,2023-07-31 21:48:45,2023-07-31 22:07:28,St Johns Pl & Washington Ave,4001.09,Degraw St & Smith St,4298.05,40.673724,-73.963161,40.682915,-73.993182,casual
16772,115B31DE3A388618,classic_bike,2023-07-31 22:32:40,2023-07-31 22:46:23,Christopher St & Greenwich St,5847.01,Washington St & Gansevoort St,6039.06,40.732916,-74.007114,40.739323,-74.008119,casual


In [4]:
# Converting Datatypes
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
print('Dataframe Info')
print(df.info())

Dataframe Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16774 entries, 0 to 16773
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ride_id             16774 non-null  object        
 1   rideable_type       16774 non-null  object        
 2   started_at          16774 non-null  datetime64[ns]
 3   ended_at            16774 non-null  datetime64[ns]
 4   start_station_name  16774 non-null  object        
 5   start_station_id    16774 non-null  object        
 6   end_station_name    16774 non-null  object        
 7   end_station_id      16774 non-null  object        
 8   start_lat           16774 non-null  float64       
 9   start_lng           16774 non-null  float64       
 10  end_lat             16774 non-null  float64       
 11  end_lng             16774 non-null  float64       
 12  member_casual       16774 non-null  object        
dtypes: datetime64[ns](2), float64(4

In [13]:
# Computing ride duration
df['ride_duration'] = round(((df['ended_at'] - df['started_at']).dt.total_seconds() / 60), 2)

In [25]:
start_lat = df['start_lat'].to_list()
start_lng = df['start_lng'].to_list()


start_lat_longs = list(set(zip(start_lat, start_lng)))

In [28]:

def get_zipcode_nom(lat, lon):
    # Nominatim endpoint for reverse geocoding
    nominatim_endpoint = "https://nominatim.openstreetmap.org/reverse"

    # Set up parameters for the request
    params = {
        "format": "json",
        "lat": lat,
        "lon": lon,
    }

    # Make the request to Nominatim
    response = requests.get(nominatim_endpoint, params=params)
    
    # Parse the JSON response
    data = response.json()

    # Extract ZIP code from the response
    if 'address' in data and 'postcode' in data['address']:
        print(lat,lon,'-->',data['address']['postcode'])
        return data['address']['postcode']
    else:
        return None

In [31]:
df['start_zip'] = df.apply(lambda row: get_zipcode_nom(row['start_lat'], row['start_lng']), axis=1)

40.76695317 -73.98169333 --> 10019
40.71600914 -73.991951823 --> 10002
40.8008363 -73.9664492472 --> 10025
40.746647 -73.993915 --> 10001
40.747066975 -73.943227291 --> 11101
40.781410700190015 -73.95595908164978 --> 10128
40.6792788 -73.98154004 --> 11217
40.69319582 -73.968919396 --> 11205
40.762009 -73.996975 --> 10036
40.663989425 -73.947569013 --> 11213
40.7208736 -73.98085795 --> 10009
40.759930849 -73.976345301 --> 10019
40.73914206 -73.979898334 --> 10010
40.75767234025041 -73.90769943594933 --> 11377
40.6843549 -73.98901629 --> 11217
40.68542400997018 -73.99127773940563 --> 11213
40.683946371 -73.976793766 --> 11217
40.72769 -73.990993 --> 10003
40.729093432 -73.998125434 --> 10012
40.74056423633952 -73.99852573871613 --> 10011
40.71221958009051 -74.01047229766846 --> 10282
40.65482 -73.973217 --> 11218
40.7859201 -73.94860294 --> 10029
40.7934337 -73.94945003 --> 10037
40.72334683218016 -73.98265913128853 --> 10009
40.69623297129316 -73.99142056703568 --> 11242
40.76541698 -7

In [32]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration,start_zip
0,3510255EC224A971,electric_bike,2023-08-01 00:37:40,2023-08-01 00:53:19,Broadway & W 58 St,6948.1,Morningside Dr & Amsterdam Ave,7741.04,40.766953,-73.981693,40.810285,-73.957365,casual,15.65,10019
1,4F68D8A99EDAC92B,classic_bike,2023-08-01 01:15:28,2023-08-01 01:30:44,Allen St & Hester St,5342.1,John St & William St,5065.04,40.716009,-73.991952,40.708621,-74.007222,member,15.27,10002
2,E5005E70D0099A96,classic_bike,2023-08-01 02:57:52,2023-08-01 03:19:44,W 106 St & Amsterdam Ave,7634.01,Madison Ave & E 120 St,7667.04,40.800836,-73.966449,40.802950,-73.942954,member,21.87,10025
3,9010CDC8C004135E,classic_bike,2023-08-01 03:14:09,2023-08-01 03:19:27,W 27 St & 7 Ave,6247.06,Lexington Ave & E 26 St,6089.08,40.746647,-73.993915,40.741459,-73.983293,member,5.30,10001
4,E526EF90BBD8CB0F,classic_bike,2023-08-01 04:41:52,2023-08-01 04:48:44,44 Dr & Jackson Ave,6352.01,36 Ave & 31 St,6572.08,40.747067,-73.943227,40.756513,-73.929575,member,6.87,11101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,33822DF82701AEBC,classic_bike,2023-07-31 19:24:53,2023-07-31 19:31:26,W 76 St & Columbus Ave,7281.09,2 Ave & E 72 St,6925.09,40.780304,-73.977716,40.768762,-73.958408,member,6.55,10023
16770,FDB8A246C88C25A5,classic_bike,2023-07-31 20:56:19,2023-07-31 21:05:05,2 Ave & E 122 St,7622.12,2 Ave & E 122 St,7622.12,40.800436,-73.934936,40.800672,-73.934900,member,8.77,10035
16771,18973BCA05D7B153,classic_bike,2023-07-31 21:48:45,2023-07-31 22:07:28,St Johns Pl & Washington Ave,4001.09,Degraw St & Smith St,4298.05,40.673724,-73.963161,40.682915,-73.993182,casual,18.72,11233
16772,115B31DE3A388618,classic_bike,2023-07-31 22:32:40,2023-07-31 22:46:23,Christopher St & Greenwich St,5847.01,Washington St & Gansevoort St,6039.06,40.732916,-74.007114,40.739323,-74.008119,casual,13.72,10014


In [33]:
df.to_csv('data/final_data/compiled_start_zip.csv')

In [40]:
# EDA
sorted_df = df.sort_values(by='ride_duration')


# Removing rows with ride duration <= 0
df = df[df['ride_duration']>0]
# Removing rows with start_zip == None
df = df[df['start_zip']!=None]

In [47]:
# Price Calculation
def calc_price(member_flag, minutes, bike_type):
    if member_flag=='casual':
        if bike_type=='classic_bike':
            if minutes<=30:
                price = 4.49
                print(member_flag, minutes, bike_type)
                return price
            else:
                price=4.49 + (0.26*(minutes - 30))
                print(member_flag, minutes, bike_type)
                return price
        elif bike_type=='electric_bike':
            price = 4.49 + (0.26 * minutes)
            print(member_flag, minutes, bike_type)
            return price
    elif member_flag == 'member':
        if bike_type=='classic_bike':
            if minutes<=45:
                price = 0
                print(member_flag, minutes, bike_type)
                return price
            else:
                price = 0.17 * minutes
                print(member_flag, minutes, bike_type)
                return price
        elif bike_type=='electric_bike':
            price = 0.17 * minutes
            print(member_flag, minutes, bike_type)
            return price

In [48]:
# calculating price for df values
df['price'] = df.apply(lambda row: calc_price(row['member_casual'], row['ride_duration'], row['rideable_type']), axis=1)

casual 15.65 electric_bike
member 15.27 classic_bike
member 21.87 classic_bike
member 5.3 classic_bike
member 6.87 classic_bike
member 14.05 electric_bike
member 5.75 classic_bike
member 7.1 classic_bike
member 11.23 classic_bike
member 11.0 classic_bike
member 15.8 classic_bike
member 13.05 classic_bike
member 10.98 classic_bike
member 5.6 classic_bike
member 2.83 classic_bike
member 24.93 classic_bike
member 12.92 classic_bike
member 6.63 classic_bike
member 6.83 classic_bike
member 8.02 classic_bike
member 2.2 classic_bike
member 12.87 classic_bike
member 5.38 classic_bike
casual 22.08 classic_bike
member 0.47 electric_bike
member 31.02 classic_bike
member 14.17 classic_bike
casual 1.98 classic_bike
member 3.63 classic_bike
member 34.07 classic_bike
member 12.62 classic_bike
member 2.42 classic_bike
member 1.63 classic_bike
member 7.05 classic_bike
member 15.05 classic_bike
member 17.32 classic_bike
member 4.83 classic_bike
casual 0.93 electric_bike
casual 8.38 classic_bike
casual 4

In [49]:
display(df)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_duration,start_zip,price
0,3510255EC224A971,electric_bike,2023-08-01 00:37:40,2023-08-01 00:53:19,Broadway & W 58 St,6948.1,Morningside Dr & Amsterdam Ave,7741.04,40.766953,-73.981693,40.810285,-73.957365,casual,15.65,10019,8.559
1,4F68D8A99EDAC92B,classic_bike,2023-08-01 01:15:28,2023-08-01 01:30:44,Allen St & Hester St,5342.1,John St & William St,5065.04,40.716009,-73.991952,40.708621,-74.007222,member,15.27,10002,0.000
2,E5005E70D0099A96,classic_bike,2023-08-01 02:57:52,2023-08-01 03:19:44,W 106 St & Amsterdam Ave,7634.01,Madison Ave & E 120 St,7667.04,40.800836,-73.966449,40.802950,-73.942954,member,21.87,10025,0.000
3,9010CDC8C004135E,classic_bike,2023-08-01 03:14:09,2023-08-01 03:19:27,W 27 St & 7 Ave,6247.06,Lexington Ave & E 26 St,6089.08,40.746647,-73.993915,40.741459,-73.983293,member,5.30,10001,0.000
4,E526EF90BBD8CB0F,classic_bike,2023-08-01 04:41:52,2023-08-01 04:48:44,44 Dr & Jackson Ave,6352.01,36 Ave & 31 St,6572.08,40.747067,-73.943227,40.756513,-73.929575,member,6.87,11101,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,33822DF82701AEBC,classic_bike,2023-07-31 19:24:53,2023-07-31 19:31:26,W 76 St & Columbus Ave,7281.09,2 Ave & E 72 St,6925.09,40.780304,-73.977716,40.768762,-73.958408,member,6.55,10023,0.000
16770,FDB8A246C88C25A5,classic_bike,2023-07-31 20:56:19,2023-07-31 21:05:05,2 Ave & E 122 St,7622.12,2 Ave & E 122 St,7622.12,40.800436,-73.934936,40.800672,-73.934900,member,8.77,10035,0.000
16771,18973BCA05D7B153,classic_bike,2023-07-31 21:48:45,2023-07-31 22:07:28,St Johns Pl & Washington Ave,4001.09,Degraw St & Smith St,4298.05,40.673724,-73.963161,40.682915,-73.993182,casual,18.72,11233,4.490
16772,115B31DE3A388618,classic_bike,2023-07-31 22:32:40,2023-07-31 22:46:23,Christopher St & Greenwich St,5847.01,Washington St & Gansevoort St,6039.06,40.732916,-74.007114,40.739323,-74.008119,casual,13.72,10014,4.490


In [50]:
df.to_csv('data/final_data/compiled_start_zip_ride_price.csv')

In [51]:
df['end_zip'] = df.apply(lambda row: get_zipcode_nom(row['end_lat'], row['end_lng']), axis=1)

40.81028506306003 -73.95736455917358 --> 10027
40.70862144 -74.00722156 --> 10038
40.80295 -73.942954 --> 10035
40.741459 -73.983293 --> 10035
40.75651272984955 -73.92957486212252 --> 11106
40.74901271 -73.98848395 --> 10120
40.6704922 -73.98541675 --> 11215
40.68683208 -73.9796772 --> 11227
40.74765947 -73.98490707 --> 10016
40.665816 -73.956934 --> 11225
40.7103231 -74.0043228 --> 10038
40.77579376683666 -73.9762057363987 --> 10023
40.75461 -73.99177 --> 10018
40.762507 -73.902534 --> 11377
40.68542400997018 -73.99127773940563 --> 11213
40.6831164 -73.99785267 --> 11231
40.663493 -73.991007 --> 11215
40.722103786686034 -73.99724900722504 --> 10012
40.72453734 -73.98185424 --> 10009
40.73275887045087 -73.99581670761108 --> 10003
40.709267 -74.013247 --> 10006
40.648005 -73.979342 --> 11218
40.782454175279874 -73.9489196240902 --> 10128
40.75934501 -73.96759673 --> 10022
40.72334683218016 -73.98265913128853 --> 10009
40.6851443 -73.95380904 --> 11216
40.764126 -73.980973 --> 10019
40.6

In [52]:
df.to_csv('data/final_data/compiled_all.csv')

In [68]:
df = pd.read_csv('data/final_data/compiled_all.csv', index_col=0)
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [69]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'ride_duration', 'start_zip', 'price', 'end_zip'],
      dtype='object')

In [70]:
# Extracting datetime features
df['start_day_of_week'] = df['started_at'].dt.day_name()
df['end_day_of_week'] = df['ended_at'].dt.day_name()


df['start_hour'] = df['started_at'].dt.hour
df['end_hour'] = df['ended_at'].dt.hour


# Extract month
df['start_month'] = df['started_at'].dt.month
df['end_month'] = df['ended_at'].dt.month


# Extract Season
# Map months to seasons
seasons = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'}
df['start_season'] = df['start_month'].map(seasons)
df['end_season'] = df['end_month'].map(seasons)

In [71]:
df.shape

(16766, 25)

In [72]:
# Filtering out extra data
df = df[df['start_day_of_week']==df['end_day_of_week']]
df.shape

(16626, 25)

In [73]:
df.to_csv('data/final_data/compiled_datetime_features.csv')

In [86]:
df['ride_duration_delta'] = pd.to_timedelta(df['ride_duration'])

# Create bins for ride durations
bins = [pd.Timedelta(minutes=0), pd.Timedelta(minutes=15), pd.Timedelta(minutes=45), pd.Timedelta(hours=1), pd.Timedelta(days=30)]
labels = ['Very Short', 'Short', 'Medium', 'Long']
df['ride_duration_category'] = pd.cut(df['ride_duration'], bins=bins, labels=labels)
display(df)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_zip,start_day_of_week,end_day_of_week,start_hour,end_hour,start_month,end_month,start_season,end_season,ride_duration_category
0,3510255EC224A971,electric_bike,2023-08-01 00:37:40,2023-08-01 00:53:19,Broadway & W 58 St,6948.1,Morningside Dr & Amsterdam Ave,7741.04,40.766953,-73.981693,...,10027.0,Tuesday,Tuesday,0,0,8,8,Summer,Summer,NaN
1,4F68D8A99EDAC92B,classic_bike,2023-08-01 01:15:28,2023-08-01 01:30:44,Allen St & Hester St,5342.1,John St & William St,5065.04,40.716009,-73.991952,...,10038.0,Tuesday,Tuesday,1,1,8,8,Summer,Summer,NaN
2,E5005E70D0099A96,classic_bike,2023-08-01 02:57:52,2023-08-01 03:19:44,W 106 St & Amsterdam Ave,7634.01,Madison Ave & E 120 St,7667.04,40.800836,-73.966449,...,10035.0,Tuesday,Tuesday,2,3,8,8,Summer,Summer,NaN
3,9010CDC8C004135E,classic_bike,2023-08-01 03:14:09,2023-08-01 03:19:27,W 27 St & 7 Ave,6247.06,Lexington Ave & E 26 St,6089.08,40.746647,-73.993915,...,10035.0,Tuesday,Tuesday,3,3,8,8,Summer,Summer,NaN
4,E526EF90BBD8CB0F,classic_bike,2023-08-01 04:41:52,2023-08-01 04:48:44,44 Dr & Jackson Ave,6352.01,36 Ave & 31 St,6572.08,40.747067,-73.943227,...,11106.0,Tuesday,Tuesday,4,4,8,8,Summer,Summer,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,33822DF82701AEBC,classic_bike,2023-07-31 19:24:53,2023-07-31 19:31:26,W 76 St & Columbus Ave,7281.09,2 Ave & E 72 St,6925.09,40.780304,-73.977716,...,10021.0,Monday,Monday,19,19,7,7,Summer,Summer,NaN
16770,FDB8A246C88C25A5,classic_bike,2023-07-31 20:56:19,2023-07-31 21:05:05,2 Ave & E 122 St,7622.12,2 Ave & E 122 St,7622.12,40.800436,-73.934936,...,10035.0,Monday,Monday,20,21,7,7,Summer,Summer,NaN
16771,18973BCA05D7B153,classic_bike,2023-07-31 21:48:45,2023-07-31 22:07:28,St Johns Pl & Washington Ave,4001.09,Degraw St & Smith St,4298.05,40.673724,-73.963161,...,11217.0,Monday,Monday,21,22,7,7,Summer,Summer,NaN
16772,115B31DE3A388618,classic_bike,2023-07-31 22:32:40,2023-07-31 22:46:23,Christopher St & Greenwich St,5847.01,Washington St & Gansevoort St,6039.06,40.732916,-74.007114,...,10014.0,Monday,Monday,22,22,7,7,Summer,Summer,NaN


In [92]:
df['ride_duration'] = df['ride_duration'] = round(((df['ended_at'] - df['started_at']).dt.total_seconds() / 60), 2)
# df.to_csv('data/final_data/compiled_datetime_features.csv')

In [94]:
df['ride_duration_delta'] = pd.to_timedelta(df['ride_duration'])

# Create bins for ride durations
bins = [pd.Timedelta(minutes=0), pd.Timedelta(minutes=15), pd.Timedelta(minutes=45), pd.Timedelta(hours=1), pd.Timedelta(days=30)]
labels = ['Very Short', 'Short', 'Medium', 'Long']
df['ride_duration_category'] = pd.cut(df['ride_duration_delta'], bins=bins, labels=labels)
display(df)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_day_of_week,start_hour,end_hour,start_month,end_month,start_season,end_season,ride_duration_category,ride_duration_new,ride_duration_delta
0,3510255EC224A971,electric_bike,2023-08-01 00:37:40,2023-08-01 00:53:19,Broadway & W 58 St,6948.1,Morningside Dr & Amsterdam Ave,7741.04,40.766953,-73.981693,...,Tuesday,0,0,8,8,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000015
1,4F68D8A99EDAC92B,classic_bike,2023-08-01 01:15:28,2023-08-01 01:30:44,Allen St & Hester St,5342.1,John St & William St,5065.04,40.716009,-73.991952,...,Tuesday,1,1,8,8,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000015
2,E5005E70D0099A96,classic_bike,2023-08-01 02:57:52,2023-08-01 03:19:44,W 106 St & Amsterdam Ave,7634.01,Madison Ave & E 120 St,7667.04,40.800836,-73.966449,...,Tuesday,2,3,8,8,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000021
3,9010CDC8C004135E,classic_bike,2023-08-01 03:14:09,2023-08-01 03:19:27,W 27 St & 7 Ave,6247.06,Lexington Ave & E 26 St,6089.08,40.746647,-73.993915,...,Tuesday,3,3,8,8,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000005
4,E526EF90BBD8CB0F,classic_bike,2023-08-01 04:41:52,2023-08-01 04:48:44,44 Dr & Jackson Ave,6352.01,36 Ave & 31 St,6572.08,40.747067,-73.943227,...,Tuesday,4,4,8,8,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16769,33822DF82701AEBC,classic_bike,2023-07-31 19:24:53,2023-07-31 19:31:26,W 76 St & Columbus Ave,7281.09,2 Ave & E 72 St,6925.09,40.780304,-73.977716,...,Monday,19,19,7,7,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000006
16770,FDB8A246C88C25A5,classic_bike,2023-07-31 20:56:19,2023-07-31 21:05:05,2 Ave & E 122 St,7622.12,2 Ave & E 122 St,7622.12,40.800436,-73.934936,...,Monday,20,21,7,7,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000008
16771,18973BCA05D7B153,classic_bike,2023-07-31 21:48:45,2023-07-31 22:07:28,St Johns Pl & Washington Ave,4001.09,Degraw St & Smith St,4298.05,40.673724,-73.963161,...,Monday,21,22,7,7,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000018
16772,115B31DE3A388618,classic_bike,2023-07-31 22:32:40,2023-07-31 22:46:23,Christopher St & Greenwich St,5847.01,Washington St & Gansevoort St,6039.06,40.732916,-74.007114,...,Monday,22,22,7,7,Summer,Summer,Very Short,0.0,0 days 00:00:00.000000013


In [96]:
df_new = df.drop(columns=['ride_duration_delta','ride_duration_new'])

In [97]:
df_new.to_csv('data/final_data/combined_time_features.csv')